# Welche Dateitypen gibt es?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [1]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [2]:
import pandas as pd

query ="MATCH (f:Git:File) RETURN f.relativePath as relativePath"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [3]:
df.head()

,relativePath
0,src/main/webapp/bower_components/bootstrap/js/...
1,src/main/webapp/bower_components/bootstrap/les...
2,src/main/webapp/bower_components/jquery/src/se...
3,src/main/webapp/bower_components/bootstrap/les...
4,src/main/webapp/bower_components/angular-resou...


Der folgende Codeabschnitt extrahiert die verschiedenen Dateitypen entsprechend der Dateiendung und zählt deren Häufigkeit. Die Dateitypen werden in der Variable <font face="Courier">datatype</font> und deren Häufigkeit in der Variable <font face="Courier">frequency</font> gespeichert.

In [4]:
# Extrahiere Dataypes aus Spalte des Dataframes.
datatypes = df['relativePath'].str.rsplit('.', 1).str[1]
# Erstelle series Objekt mit zählen der einzelnen Datentypen zugeordnet den einzelnen Dateitypen.
series = datatypes.value_counts()
# Umwandeln der series Spalten in list Objekte.
datatype = list(series.index)
frequency = list(series)
print(datatype)
print(frequency)
# Erzeuge die Kategorie "andere", in der alle Dateitypen gesammelt werden, die weniger oder genau 20 mal auftauchen
andere = 0
for wert in frequency[:]:
    index = frequency.index(wert)
    if wert <= 20:
        andere += wert
        datatype.remove(datatype[index])
        frequency.remove(wert)
frequency.append(andere)
datatype.append("andere")




['js', 'png', 'java', 'css', 'less', 'html', 'xml', 'json', 'jsp', 'md', 'gif', 'txt', 'php', 'adoc', 'properties', 'mustache', 'prefs', 'jpg', 'map', 'tag', 'gitignore', 'svg', 'jshintrc', 'graphml', 'yml', 'sql', 'woff', 'ttf', 'eot', 'plantuml', 'classpath', 'springBeans', 'ico', 'project', 'component', 'editorconfig', 'xsd', 'name', 'jmx', 'jshintignore', 'csslintrc', 'scss', 'bowerrc', 'iml', 'pdf', 'jsdtscope', 'config', 'gitmodules', 'dtd', 'container', 'jar', 'mailmap', 'log', 'cmd', 'tern-project', 'gitattributes', 'coffee', 'gzip', 'ts', 'woff2']
[1277, 475, 225, 133, 127, 108, 82, 68, 39, 35, 33, 32, 23, 19, 18, 18, 14, 13, 13, 9, 6, 6, 6, 5, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).

## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).



In [5]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [6]:
import pygal 
pie_chart = pygal.Pie()
pie_chart.title = 'Dateitypen'
for einzelneDateitypen in datatype:
    index= datatype.index(einzelneDateitypen)
    anzahl=frequency[index]
    pie_chart.add(einzelneDateitypen, anzahl)
display(HTML(base_html.format(rendered_chart=pie_chart.render(is_unicode=True))))